In [1]:
import pandas as pd
from Person import *

In [3]:
df = pd.read_excel("4-TableS1_augmented_with_FAO_data.xlsx").dropna()
df.head()

,Product,Type,RetailUnit,Comment,kcalPerRetailUnit,gProteinPerRetailUnit,gFatPerRetailUnit,gCarbPerRetailUnit
0,Wheat & Rye (Bread),CarbSource,kg,1 kg,2490.0,82.0,12.0,514.10
1,Maize (Meal),CarbSource,kg,1 kg of meal (for polenta),3630.0,84.0,12.0,797.10
2,Barley (Beer),Extra,L,1 liter of beer,499.8,5.1,0.0,119.85
3,Oatmeal,CarbSource,kg,1 kg of rolled oats,3840.0,160.0,63.0,661.40
4,Rice,CarbSource,kg,1 kg of full grain white or brown rice (dry),3600.0,67.0,7.0,817.60


In [4]:
# Calculate needs for a person
will = Person("male", 62, 173, 22, '2')
cal_needed_per_day = will.calculate_needs()
cal_for_lunch = cal_needed_per_day * 0.4
cal_for_diner = cal_needed_per_day * 0.3

In [5]:
types = df['Type'].unique()
types_weight = {'Vegetable': 0.1, 'Fruit': 0.075, 'CarbSource': 0.50, 'ProteinSource': 0.2, 'FatSource': 0.10, 'Extra': 0.025}

In [6]:
def generate_meal(cal):
    meal = dict(zip(types, ('', '')))
    for type in types:
        random_food = df[df['Type'] == type][['Product', 'kcalPerRetailUnit', 'RetailUnit']].sample(1).values
        meal[type] = (random_food[0][0], f"{cal * types_weight[type] / random_food[0][1]} {random_food[0][2]}")
    
    return meal

In [7]:
print("REPAS POUR 7 JOURS")
print("\n\n")
for i in range(7):
    print(f"Jour {i+1}")
    print("===============")
    print(f"Déjeuner {generate_meal(cal_for_lunch)}")
    print(f"Dinner {generate_meal(cal_for_diner)}")
    print("\n\n")

REPAS POUR 7 JOURS



Jour 1
Déjeuner {'CarbSource': ('Cassava', '0.4393348623853211 kg'), 'Extra': ('Dark Chocolate', '0.006092557251908397 kg'), 'ProteinSource': ('Bovine Meat (beef herd)', '0.1277 kg'), 'FatSource': ('Olive Oil', '0.01182991600790514 L'), 'Vegetable': ('Other Vegetables', '0.4353409090909091 kg'), 'Fruit': ('Apples', '0.14964843749999998 kg')}
Dinner {'CarbSource': ('Rice', '0.099765625 kg'), 'Extra': ('Barley (Beer)', '0.03592999699879952 L'), 'ProteinSource': ('Lamb & Mutton', '0.12072478991596639 kg'), 'FatSource': ('Rapeseed Oil', '0.008872437005928853 L'), 'Vegetable': ('Brassicas', '0.37805921052631575 kg'), 'Fruit': ('Other Fruit', '0.11971875 kg')}



Jour 2
Déjeuner {'CarbSource': ('Wheat & Rye (Bread)', '0.19231927710843374 kg'), 'Extra': ('Coffee', '0.04275669642857143 kg'), 'ProteinSource': ('Soymilk', '0.34439050701186624 L'), 'FatSource': ('Rapeseed Oil', '0.01182991600790514 L'), 'Vegetable': ('Other Vegetables', '0.4353409090909091 kg'), 'Fruit': ('C

In [8]:
import math

def gaussian(x, mu, sigma):
    return math.exp(-((x - mu) ** 2) / (2 * sigma ** 2))

def fitness(kcal, prot, fat, carb):
    # Calculer la fitness en utilisant des fonctions gaussiennes autour des cibles
    kcal_fitness = gaussian(kcal, 800, sigma=50)
    prot_fitness = gaussian(prot, 25, sigma=5)
    fat_fitness = gaussian(fat, 30, sigma=5)
    carb_fitness = gaussian(carb, 100, sigma=10)

    # La fitness finale est la moyenne des fitness pour chaque nutriment
    fitness_score = (kcal_fitness + prot_fitness + fat_fitness + carb_fitness) / 4

    return fitness_score

# Test de la fonction
print(fitness(00, 25, 3, 100))


0.5000001163928929


## Access different values of an aliment

In [10]:
df.head()

,Product,Type,RetailUnit,Comment,kcalPerRetailUnit,gProteinPerRetailUnit,gFatPerRetailUnit,gCarbPerRetailUnit
0,Wheat & Rye (Bread),CarbSource,kg,1 kg,2490.0,82.0,12.0,514.10
1,Maize (Meal),CarbSource,kg,1 kg of meal (for polenta),3630.0,84.0,12.0,797.10
2,Barley (Beer),Extra,L,1 liter of beer,499.8,5.1,0.0,119.85
3,Oatmeal,CarbSource,kg,1 kg of rolled oats,3840.0,160.0,63.0,661.40
4,Rice,CarbSource,kg,1 kg of full grain white or brown rice (dry),3600.0,67.0,7.0,817.60


In [27]:
from collections import namedtuple

Aliment = namedtuple('Aliment', ['name', 'kcal', 'prot', 'fat', 'carb'])
aliments= []

grammes = [5, 50, 100, 250]
for index, row in df.iterrows():
    for i in range(len(grammes)):
        aliments.append(Aliment(row['Product'] + f" {grammes[i]}g", row['kcalPerRetailUnit'] * grammes[i]/1000, row['gProteinPerRetailUnit'] * grammes[i]/1000, row['gFatPerRetailUnit'] * grammes[i]/1000, row['gCarbPerRetailUnit'] * grammes[i]/1000))

aliments

[Aliment(name='Wheat & Rye (Bread) 5g', kcal=12.45, prot=0.41, fat=0.06, carb=2.5705),
 Aliment(name='Wheat & Rye (Bread) 50g', kcal=124.5, prot=4.1, fat=0.6, carb=25.705),
 Aliment(name='Wheat & Rye (Bread) 100g', kcal=249.0, prot=8.2, fat=1.2, carb=51.41),
 Aliment(name='Wheat & Rye (Bread) 250g', kcal=622.5, prot=20.5, fat=3.0, carb=128.525),
 Aliment(name='Maize (Meal) 5g', kcal=18.15, prot=0.42, fat=0.06, carb=3.9855),
 Aliment(name='Maize (Meal) 50g', kcal=181.5, prot=4.2, fat=0.6, carb=39.855),
 Aliment(name='Maize (Meal) 100g', kcal=363.0, prot=8.4, fat=1.2, carb=79.71),
 Aliment(name='Maize (Meal) 250g', kcal=907.5, prot=21.0, fat=3.0, carb=199.275),
 Aliment(name='Barley (Beer) 5g', kcal=2.499, prot=0.0255, fat=0.0, carb=0.59925),
 Aliment(name='Barley (Beer) 50g', kcal=24.99, prot=0.25499999999999995, fat=0.0, carb=5.9925),
 Aliment(name='Barley (Beer) 100g', kcal=49.98, prot=0.5099999999999999, fat=0.0, carb=11.985),
 Aliment(name='Barley (Beer) 250g', kcal=124.95, prot=1.2

In [40]:
gaussian(80000, 800, 10000)

2.394061514095669e-14